⚠️ Need to have .env file in the project root with OPENAI_API_KEY={your_openai_key}

In [ ]:
!pip install langchain openai python-dotenv

In [5]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import LLMChain

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

chat = ChatOpenAI(temperature = 0, openai_api_key=openai_api_key)
system_message_prompt = SystemMessagePromptTemplate.from_template_file('data/system_prompt.txt', [])
with open('data/email_thread.txt', 'r') as file:
    thread = file.read()

with open('data/incident_report.txt', 'r') as file:
    report = file.read()

email_template = """The following is the email thread discussing the incident:
{thread}"""

report_template = """The following is the corresponding incident report:
{report}"""

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt]
    + HumanMessagePromptTemplate.from_template(email_template).format_messages(thread = thread)
    + HumanMessagePromptTemplate.from_template(report_template).format_messages(report = report)
    + [HumanMessage(content = 'Summarize the incident report and email thread in readable and concise way, so that I could easily understand the incident root cause and the current status')]
)

conversation = LLMChain(prompt=chat_prompt, llm=chat)

response = conversation.predict()

print(response)

Summary:

On July 1, 2023, a critical incident occurred in the CorePay application, impacting payment processing. The issue was identified as a bug in the payment validation process. The development team promptly developed and deployed a fix within 6 hours. The incident has been resolved, and payment processing functionality has been restored. The incident report recommends strengthening code review and testing processes, enhancing application monitoring, and implementing a rollback strategy for future releases.

Email Thread Summary:

- Jason Bourne, a production support engineer, alerted Linda Reed, the production support manager, about an issue in the CorePay application affecting payment processing.
- Linda informed Kevin Murray, the application development manager, who mobilized his team to investigate the code changes in the latest release.
- Neil Patel, an application developer, discovered an unusual change related to payment validations and suspected it to be the cause of the i